# Neo4J - with Python #
##### Lukas Hinterleitner, Daniel Nepp #####
For a complete documentation visit: https://neo4j.com/developer/python/

---

## Installation ##
### 1. Download and install Neo4j Desktop: ###
https://neo4j.com/download/

Before you can download the installation file you have to provide your data.
If you don't want give your data away, you can type in random stuff :)

An activation key gets displayed while downloading (shown in the picture below).
Keep it, you will need it later during the installation.
<br><br>

![Neo4J Activation Key](screenshots/key.png)
<br><br>

After the installation finished you should see the home screen which is displayed in the image below.
We will get back to it later when we create our first Neo4J graph database.

![Home screen Neo4J Desktop](screenshots/home_screen.png)
<br><br>

### 2. Create a Conda-Environment : ###

`$ conda create --name neo4j python=3.8.2`


### 3. Activate Conda-Environment : ###

`$ conda activate neo4j`


### 4. Install the Neo4j driver : ###
https://anaconda.org/conda-forge/neo4j-python-driver

`$ conda install -c conda-forge neo4j-python-driver`

### 5. Install jupyter notebook : ###

`$ conda install -c anaconda jupyter`

or install jupyter lab

`$ conda install -c conda-forge jupyterlab`

or install jupyter in pycharm...


## Creating Neo4J Database : ##

Description...

![Home screen Neo4J Desktop](screenshots/create_local_database.png)
<br><br>

Description...

![Home screen Neo4J Desktop](screenshots/create_database.png)
<br><br>

Description...

![Home screen Neo4J Desktop](screenshots/database_after_installation.png)
<br><br>

Description...

![Home screen Neo4J Desktop](screenshots/database_settings.png)
<br><br>

Description...

![Home screen Neo4J Desktop](screenshots/neo4j_browser_home_screen.png)
<br><br>

---

## Using Neo4j : ##

The code above is a Neo4J Python hello world example! :)


In [6]:
from neo4j import GraphDatabase

uri, user, password = 'bolt://localhost:7687', 'neo4j', 'neo4j_'

def _create_and_return_greeting(tx, message):
    result = tx.run("CREATE (a:Greeting) "
                    "SET a.message = $message "
                    "RETURN a.message + ', from node ' + id(a)", message=message)
    return result.single()[0]

driver = GraphDatabase.driver(uri, auth=(user, password))

with driver.session() as session:
    greeting = session.write_transaction(_create_and_return_greeting, "Hello World!")
    print(greeting)

driver.close()

Hello World!, from node 0


### 1. Create Neo4J Entities : ###

```sql
CREATE (
        TheMatrix:Movie
        {title:'The Matrix', released:1999, tagline:'Welcome to the Real World'}
        )
```

```sql
CREATE (
        Keanu:Person
        {name:'Keanu Reeves', born:1964}
        )
```

### 2. Create Neo4J Relations : ###

```sql
(Keanu) - [:ACTED_IN { roles : [ 'Neo' ]}] -> (TheMatrix)
```

```sql
(LillyW) - [:DIRECTED] -> (TheMatrix)
```

### 3. Find Neo4J Entities : ###

Find Keanu Reeves - returns single entity:
```sql
MATCH (keanu {name: "Keanu Reeves"}) RETURN keanu
```

Find The Matrix - returns single entity:
```sql
MATCH (theMatrix {title: "The Matrix"}) RETURN theMatrix
```

Find ten people - returns ten entities:
```sql
MATCH (people:Person) RETURN people.name LIMIT 10
```

Find movies produced in the 90's - returns all movies between 1990 and 2000:
```sql
MATCH (nineties:Movie) WHERE nineties.released >= 1990 AND nineties.released < 2000 RETURN nineties.title
```

List all Keanu Reeves movies - returns all movies where Keanu Reeves acted in:
```sql
MATCH (keanu:Person {name: "Keanu Reeves"}) - [:ACTED_IN] -> (keanuMovies) RETURN keanu,keanuMovies
```

Who directed "The Matrix"? - returns all directors of The Matrix:
```sql
MATCH (theMatrix {title: "The Matrix"}) <- [:DIRECTED] - (directors) RETURN directors.name
```

Keanu Reeves' co-actors -- returns all actors who acted in any movie aside Keanu Reeves:
```sql
MATCH (keanu:Person {name:"Keanu Reeves"}) - [:ACTED_IN] -> (m) <- [:ACTED_IN] - (coActors) RETURN coActors.name
```

How people are related to "The Matrix" - returns all people inclusive their relations to the movie The Matrix:
```sql
MATCH (people:Person) - [relatedTo] - (:Movie {title: "The Matrix"}) RETURN people.name, Type(relatedTo), relatedTo
```

Movies and actors up to 4 "hops" away from Kevin Bacon:
```sql
MATCH (bacon:Person {name:"Kevin Bacon"}) - [*1..4] - (hollywood)
RETURN DISTINCT hollywood
```
Note: hollywood here is just a placeholder for any entity

Bacon path, the shortest path of any relationships to Meg Ryan:
```sql
MATCH p=shortestPath(
(bacon:Person {name:"Kevin Bacon"}) - [*] - (meg:Person {name:"Meg Ryan"})
)
RETURN p
```

Extend Tom Hanks co-actors, to find co-co-actors who haven't worked with Tom Hanks:
```sql
MATCH (tom:Person {name:"Tom Hanks"})-[:ACTED_IN]->(m)<-[:ACTED_IN]-(coActors),
    (coActors)-[:ACTED_IN]->(m2)<-[:ACTED_IN]-(cocoActors)
WHERE NOT (tom)-[:ACTED_IN]->()<-[:ACTED_IN]-(cocoActors) AND tom <> cocoActors
RETURN cocoActors.name AS Recommended, count(*) AS Strength ORDER BY Strength DESC
```

Find someone to introduce Tom Hanks to Tom Cruise:
```sql
MATCH (tom:Person {name:"Tom Hanks"})-[:ACTED_IN]->(m)<-[:ACTED_IN]-(coActors),
  (coActors)-[:ACTED_IN]->(m2)<-[:ACTED_IN]-(cruise:Person {name:"Tom Cruise"})
RETURN tom, m, coActors, m2, cruise
```

### 4. Delete Neo4J Graph : ###
Delete all Movie and Person nodes, and their relationships:
```sql
MATCH (n) DETACH DELETE n
```

Note you only need to compare property values like this when first creating relationships
Prove that the Movie Graph is gone
```sql
MATCH (n) RETURN n
```


In [ ]:
... the above is a neo hello world example! :)


... the above is a neo hello world example! :)
